In [1]:
import pandas as pd
import numpy as np

df_sex_age = pd.read_excel(r"C:\Users\ubunt\OneDrive\Pulpit\Biohackaton\Otylosc\Pheno_SEX_AGE_ZAKODOWANE.xlsx")
df_log_bmi = pd.read_excel(r"C:\Users\ubunt\OneDrive\Pulpit\Biohackaton\Otylosc\Pheno_log_BMI_zakodowany.xlsx")
pedtry = pd.read_csv(r"C:\Users\ubunt\OneDrive\Pulpit\Biohackaton\Otylosc\BioNinjaHack_obesity.ped",sep=" ",header=None)
snipy = pd.read_csv(r"C:\Users\ubunt\OneDrive\Pulpit\Biohackaton\Otylosc\BioNinjaHack_obesity.bim",sep="\t",header=None)


df_age = df_sex_age.AGE
df_log_bmi.insert(2, "Age", df_age, True)

df_polaczony = pd.merge(df_sex_age,df_log_bmi,on="IID")
df_polaczony = df_polaczony.drop(columns='SEX_y')
df_polaczony['index_bmi'] = df_polaczony['log_BMI'].apply(lambda x:2 if x>= 1.397940 else 1 )

pedtry.rename(columns={5:'AAlog_BMI',0:'FID', 1:'IID',2:'M',3:'F',4:'Gender'}, 
                 inplace=True)

pedtry['AAlog_BMI'] = df_polaczony['log_BMI']
pedtry['AGE'] = df_polaczony['AGE']

pedtry['AGE0_20'] = 0
pedtry['AGE20-40'] = 0
pedtry['AGE40plus'] = 0

pedtry.loc[pedtry['AGE'] < 20, 'AGE0_20'] = 1
pedtry.loc[(pedtry['AGE'] >=20) & (pedtry['AGE'] <=40) , 'AGE20-40'] = 1
pedtry.loc[pedtry['AGE'] > 40, 'AGE40plus'] = 1

snipy['Check'] = snipy[4]+snipy[5]
snipy.drop(columns=[2,3])
snipy.head(5)

snipnazwa = snipy[1].tolist()

for i in range(0,658,2):
    pedtry[snipnazwa[int(i/2)]] = pedtry[6+i]+pedtry[7+i]
    
    
colList = ['FID', 'IID','M','F','Gender', 'AAlog_BMI']

colList2 = list(pedtry.columns)
colList2 = colList2[664:]
colList.extend(colList2)
pedtry = pedtry[colList]


pedtry.head(5)




,FID,IID,M,F,Gender,AAlog_BMI,AGE,AGE0_20,AGE20-40,AGE40plus,...,rs837550,rs2287072,rs112426189,rs3744374,rs12602590,rs11654604,rs200805689,rs117651561,rs79742527,rs143040759
0,3,1,0,0,2,1.410102,64,0,0,1,...,AA,AA,TG,AG,CC,GG,TT,GG,GG,GG
1,4,2,0,0,2,1.391112,29,0,1,0,...,GG,AA,GG,GG,CC,AG,TT,GG,GG,GG
2,5,3,0,0,1,1.381837,64,0,0,1,...,GA,AA,GG,AA,CC,GG,TT,GG,GG,GG
3,6,4,0,0,2,1.290702,21,0,1,0,...,GG,AA,GG,GG,CC,GG,TT,GG,GG,GG
4,7,5,0,0,2,1.494572,38,0,1,0,...,GA,AA,TG,AG,CC,GG,TT,GG,GG,GG


In [2]:
pedtry = pedtry.drop(columns=['M','F'])

listaSnipow = snipy.Check.tolist()

columns = snipnazwa
pedtry.loc[:, columns] = pedtry.loc[:, columns].isin(set(listaSnipow))


pedtry = pedtry.applymap(lambda x:1 if x == True else x)
pedtry = pedtry.applymap(lambda x:0 if x == False else x)

pedtry = pedtry.where(pedtry.AAlog_BMI != -9).dropna()

pedtry['Gender'] = pedtry['Gender'] - 1


newColList = pedtry.columns.tolist()
newColList.sort()

pedtry = pedtry[newColList]
pedtry = pedtry.drop(columns=['FID','IID'])

import os
cwd = os.getcwd()

import math
normalna = math.log(24.99, 10)
nadwaga = math.log(29.99, 10)

pedtry['AAlog_BMI'] = pedtry['AAlog_BMI'].apply(lambda x:0 if x <= normalna else x)
pedtry['AAlog_BMI'] = pedtry['AAlog_BMI'].apply(lambda x:1 if (x > normalna) and (x <= nadwaga) else x)
pedtry['AAlog_BMI'] = pedtry['AAlog_BMI'].apply(lambda x:2 if x > nadwaga else x)

pedtry = pedtry.astype(int)

pedtry = pedtry.drop(columns='AGE')

pedtry.to_csv("finalFileClass.csv", sep = ';')


In [9]:
pedtry.AAlog_BMI.value_counts()

0    2855
1    1861
2     827
Name: AAlog_BMI, dtype: int64